In [12]:
import numpy as np
import pandas as pd

s = pd.Series({'Google':[.2,.3], 'Apple': [.3,.1],'Ebay':[-.1,-.2]})
x=s.values

# x = np.array([[0, 2 , 3], [1, 1, 6], [2, 0, 9]]).T

# np.cov(x)
# s = pd.Series({'a':[1,2], 'b':[2,3], 'c':[3,5]})
# u=np.array(s.values)
# u.astype(int)
# print(list(s.values))
returns=np.array(list(s.values))
cov_matrix=np.cov(np.array(list(s.values)))
print(cov)

[[ 0.02   0.01  -0.01 ]
 [ 0.01   0.005 -0.005]
 [-0.01  -0.005  0.005]]


In [13]:
# ticker= pd.Series({'Google':[1,2,3], 'Apple': [3,6,9], 'AMZN':[8,10,12]})
# cov_matrix=np.cov(np.array(list(ticker.values)))

In [44]:
diag=cov_matrix.diagonal()
diag_matrix=np.diag(np.diag(cov_matrix)) # gives n*n diagonal matrix
diag_matrix

array([[ 0.02 ,  0.   ,  0.   ],
       [ 0.   ,  0.005,  0.   ],
       [ 0.   ,  0.   ,  0.005]])

In [15]:
from arch import arch_model
garch11 = arch_model(diag, p=1, q=1)
res = garch11.fit(update_freq=10)
print(res.summary())

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -10.7140986901
            Iterations: 9
            Function evaluations: 58
            Gradient evaluations: 9
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                      y   R-squared:                      -0.034
Mean Model:             Constant Mean   Adj. R-squared:                 -0.034
Vol Model:                      GARCH   Log-Likelihood:                10.7141
Distribution:                  Normal   AIC:                          -13.4282
Method:            Maximum Likelihood   BIC:                          -17.0337
                                        No. Observations:                    3
Date:                Thu, Feb 09 2017   Df Residuals:                       -1
Time:                        16:43:59   Df Model:                            4
                                  Mean Model                                  
     

In [22]:
alpha=res.params[2]
beta=res.params[3]
# std_err=np.random.normal(0, 1, 3) #epselon is the returns 

In [45]:
np.linalg.inv(diag_matrix)

array([[  50.,    0.,    0.],
       [   0.,  200.,    0.],
       [   0.,    0.,  200.]])

In [60]:
# esubt=np.linalg.inv(diag_matrix)
ret=[.1,.2,.13] # just to see how it works
esubt=np.matmul(np.linalg.inv(diag_matrix),returns)
# esubt=np.matmul(np.linalg.inv(diag_matrix),returns)
esubt

array([[ 15.,   5.],
       [-20., -40.],
       [ 40.,  60.]])

In [62]:
#not sure
qdash=np.cov(esubt)
q_start=np.ones((3,3))
qdash
# q_start

array([[  50.,  100., -100.],
       [ 100.,  200., -200.],
       [-100., -200.,  200.]])

In [63]:
q=((1-alpha-beta)*qdash) + (alpha*np.matmul(esubt,esubt.T)) + ((beta)*q_start)
q

array([[ 12.76413125,  24.7683468 , -23.24851542],
       [ 24.7683468 ,  48.77677791, -47.25694654],
       [-23.24851542, -47.25694654,  48.77677791]])

In [73]:
# r=np.matmul(np.linalg.inv(qdash),q)
qstar=np.diag(np.diag(cov_matrix))
qstar=np.sqrt(qstar)
qstar

array([[ 0.14142136,  0.        ,  0.        ],
       [ 0.        ,  0.07071068,  0.        ],
       [ 0.        ,  0.        ,  0.07071068]])

In [76]:
r=np.matmul(np.linalg.inv(qstar),q,np.linalg.inv(qstar))
r

array([[  90.25603759,  175.13865982, -164.39182908],
       [ 350.27731964,  689.80780854, -668.31414707],
       [-328.78365817, -668.31414707,  689.80780854]])

In [78]:
predi_cov=np.matmul(diag_matrix,r,diag_matrix)
predi_cov

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])